In [ ]:
!pip install polars

In [ ]:
import polars as pl
import numpy as np
import implicit
from scipy.sparse import csr_matrix

In [ ]:
df_train = pl.read_parquet('input/Otto_Full_Optimized_Memory_Footprint/train.parquet')
df_test = pl.read_parquet('input/Otto_Full_Optimized_Memory_Footprint/test.parquet')
# df_train = pl.read_parquet('input/otto_train_and_test_data_for_local_validation/train.parquet')
# df_test = pl.read_parquet('input/otto_train_and_test_data_for_local_validation/test.parquet')
df = pl.concat([df_train, df_test])

In [ ]:
naids = df['aid'].max() + 1
nsessions = df['session'].max() + 1
naids, nsessions

In [ ]:
type_weight = {0:1, 1:2, 2:4}
df = df.drop('ts').unique(subset=['session', 'aid', 'type'])
df = df.with_column(pl.col('type').apply(lambda x:type_weight[x]).alias('wgt'))

In [ ]:
df = df.with_column(pl.col('wgt').cast(pl.UInt8))
df = df.groupby(['session', 'aid']).agg(pl.col('wgt').sum())

In [ ]:
df = df.sort(['aid', 'session'])

In [ ]:
df

In [ ]:
row = df['aid'].to_numpy()
col = df['session'].to_numpy()
data = df['wgt'].to_numpy()

In [ ]:
csr = csr_matrix((data, (row, col)), shape=(naids, nsessions), dtype=np.int32)

In [ ]:
# model = implicit.bpr.BayesianPersonalizedRanking(factors=100, num_threads=4, iterations=300)
# model = implicit.als.AlternatingLeastSquares(factors=50, num_threads=16, iterations=15)
model = implicit.lmf.LogisticMatrixFactorization(factors=100, num_threads=16, iterations=90)

In [ ]:
%%time
model.fit(csr)

In [ ]:
import gc
del df
gc.collect()

In [ ]:
!mkdir input/embedding_lmf_test

In [ ]:
np.save('input/embedding_lmf_test/user_factors.npy', model.user_factors)
np.save('input/embedding_lmf_test/item_factors.npy', model.item_factors)